In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from gen_graph import gen_graph

In [6]:
# lsit all folders
import os
path = '/home/dan/CEIA/HD2/CEIA/evah/test'
folders = os.listdir(path)
for f in folders:
    if 'test_' in f:
        p = os.path.join(path, f)
        print(p)
        gen_graph( pd.read_csv(p+"/result.csv"),
                               p+"/dashboard.png" )
        

/home/dan/CEIA/HD2/CEIA/evah/test/test_2024-08-19_22-27-31
/home/dan/CEIA/HD2/CEIA/evah/test/test_2024-08-20_15-19-02
/home/dan/CEIA/HD2/CEIA/evah/test/test_2024-08-20_15-40-29
/home/dan/CEIA/HD2/CEIA/evah/test/test_2024-08-22_10-55-12


In [7]:
teste = "/home/dan/CEIA/HD2/CEIA/evah/test_2024-08-23_10-02-31"
gen_graph( pd.read_csv(teste+"/result.csv"), teste+"/dashboard.png" )


In [ ]:
p = test_2024-08-23_10-02-31

In [46]:
df.y_hat.unique()

array(['RECUSADO', 'Default',
       'NÃO ENCONTRADO DOCUMENTOS QUE AUXILIEM A DECISÃO', 'AUTORIZADO'],
      dtype=object)

In [85]:

colors = { 'AUTORIZADO': 'green', 
            'RECUSADO': 'red', 
            'NÃO ENCONTRADO DOCUMENTOS QUE AUXILIEM A DECISÃO': '#FFA500',
            'Default': '#FFA500'
        }


def make_pie(ax, serie):
        # sort series by index
        serie = serie.sort_index()
        
        _, texts, _ = ax.pie( serie,  
                              labels=serie.index,  
                                        autopct='%1.1f%%',  
                              colors=serie.index.map(colors).values)
        for text in texts:
            if ('AUTORIZADO' in text.get_text()) or ('RECUSADO' in text.get_text()):
                text.set_fontweight('bold')  
                text.set_color(colors[text.get_text()])                         
                text.set_fontsize(12)
        
        ax.set_ylabel('')


def gen_graph(df_input, path_To_save):        


    df = df_input.copy()
    
    dict_status = {True: 'AUTORIZADO', False: 'RECUSADO'}

    df.STATUS = df.STATUS.map(dict_status)
    df['acerto'] = df.STATUS == df.y_hat
    df['erro'] = df.STATUS != df.y_hat
    df['sem_doc_or_codigo'] = (df.y_hat == 'NÃO ENCONTRADO DOCUMENTOS QUE AUXILIEM A DECISÃO') | (df.y_hat == 'Default')
    ######
    
    
    fig, axs = plt.subplots(4, 2, figsize=(20, 15))
    
    ################# 0, 0
    ax = axs[0][0]
    acuracia = df.acerto.sum() / df.shape[0]

    # horizontal bar plot
    ax.barh(0,df.shape[0], color='gray', label='Total', edgecolor='black',alpha=0.4)
    ax.barh(0,df.acerto.sum(), color='green', label='Acertos')

    ax.legend()
    ax.set_title(f'Acurácia do modelo - {acuracia:.2f}')
    ax.set_yticks([])





    ################# 0, 1
    ax = axs[0][1]
    acuracia = (df.sem_doc_or_codigo.sum() / df.shape[0]) * 100
    ax.barh(0,df.shape[0], color='gray', label='Total',alpha=0.4)
    ax.barh(0,df['sem_doc_or_codigo'].sum()+df.acerto.sum(), color='green', label='Acertos')
    
    ax.barh(0,df['sem_doc_or_codigo'].sum(), color='#FFA500', label='Sem Documentos ou Código')

    ax.legend()
    ax.set_title(f'Sem códigou ou sem informações - {acuracia:.2f} %')
    ax.set_yticks([])


    ################# 1, 0 -  dos itens Autorizados, quais foram autorizados?
    ax = axs[1][0]
    df_autorizado = df[df.STATUS == 'AUTORIZADO']
    df_autorizado = df_autorizado.y_hat.value_counts()
    make_pie(ax, df_autorizado)
    ax.set_title('Apenas Itens Autorizados ( {} itens )'.format(df_autorizado.sum()))
    
    

    ################# 1, 1 - Dos itens Autorizados respondidos, quais foram autorizados?
    ax = axs[1][1]
    df_autorizado = df [ (df.STATUS == 'AUTORIZADO') & ((df.y_hat == 'AUTORIZADO') | (df.y_hat == 'RECUSADO'))]
    df_autorizado = df_autorizado.y_hat.value_counts()
    make_pie(ax, df_autorizado)
    ax.set_title('Dos itens Autorizados, quantos foram respondidos?( {} itens )'.format(df_autorizado.sum()))
    


    #######################################
    #################### ITENS NEGADOS
    #######################################


    ################# 2, 0 - dos itens Negados, quantos foram realmente negados?
    ax = axs[2][0]
    df_recusado = df[df.STATUS == 'RECUSADO']
    df_recusado = df_recusado.y_hat.value_counts()
    make_pie(ax, df_recusado)
    ax.set_title('Apenas Itens recusados ({} itens )'.format(df_recusado.sum()))






    # Dos itens REJEITADOS respondidos, quais foram autorizados?
    ax = axs[2][1]
    df_recusado = df [ (df.STATUS == 'RECUSADO') & ((df.y_hat == 'AUTORIZADO') | (df.y_hat == 'RECUSADO'))]
    df_recusado = df_recusado.y_hat.value_counts()
    make_pie(ax, df_recusado)
    ax.set_title('Dos itens recusados, quantos foram respondidos?( {} itens )'.format(df_recusado.sum()))
    

    # matriz de confusao
    ax = axs[3][0]
    respondidos = df[ (df.y_hat == 'AUTORIZADO') | (df.y_hat == 'RECUSADO')]
    confusion_matrix = pd.crosstab(respondidos.STATUS, respondidos.y_hat)
    
    confusion_matrix.plot(kind='bar', ax=ax, color=[colors[x] for x in confusion_matrix.columns],
                            edgecolor='black')
    
    ax.set_title(f'Matriz de Confusão - apenas dos itens considerados como respondidos ( {respondidos.shape[0]} itens )')
    #gira o label x text em 90º
    ax.xaxis.set_label_text('Status do Item', fontsize=12, fontweight='bold')
    
    
    ax.legend(title='Classificação do Modelo')
    
    
    # remover o ultimo axis
    axs[3][1].axis('off')
    # colocar o pe nultimo axis no centro da figura
    fig.delaxes(axs[3][1])
    
        

    fig.tight_layout()

    fig.savefig(path_To_save)
    plt.close(fig)
        



a = gen_graph(df, '/home/dan/CEIA/HD2/CEIA/evah/test/test_2024-08-22_10-55-12/dashboard.png')
a

In [15]:

a_cor = a.map(colors)

Index(['blue', 'blue', 'red', 'green'], dtype='object', name='y_hat')